In [1]:
import numpy as np
import pandas as pd

%load_ext rpy2.ipython

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import os

def load_smooth_dev():
    """Load the smooth package in development mode"""
    smooth_path = "/home/filtheo/smooth/"
    
    ro.r(f'''
    if (!requireNamespace("devtools", quietly=TRUE)) {{
        install.packages("devtools", repos="https://cran.rstudio.com/")
    }}
    devtools::load_all("{smooth_path}")
    ''')
    
    print("Smooth package loaded in development mode")

load_smooth_dev()


ℹ Loading smooth


R[write to console]: Loading required package: greybox

R[write to console]: Package "greybox", v2.0.6 loaded.


R[write to console]: This is package "smooth", v4.3.2.41006


R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/lib/R/site-library’ contains no packages



Smooth package loaded in development mode


# Two-Stage Initialization Tests - R Implementation

Two-stage initialization works by:
1. First running a model with `initial="two-stage"` (full backcasting) to get good starting values
2. Using those values as initial guesses for optimization, allowing parameter refinement

This should produce results that are:
- Different from pure backcasting (since parameters are refined)
- Different from pure optimal (since starting values are better)
- Generally better or similar quality to both methods


### Test 1: Global level ETS(A,N,N) - two-stage vs optimal vs backcasting


In [2]:
np.random.seed(33)
n_points = 100
time_series = np.random.normal(100, 10, n_points)
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [3]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'optimal')
cat('ETS(A,N,N) with optimal initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)


ETS(A,N,N) with optimal initial:
Parameters: 0.02502502 98.50918 
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 101.134 101.134 101.134 101.134 101.134 101.134 101.134 101.134 101.134
[10] 101.134 101.134 101.134


In [4]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'backcasting', nIterations = 2)
cat('ETS(A,N,N) with backcasting (nIterations=2):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,N,N) with backcasting (nIterations=2):
Parameters: 0 
$mean
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911
 [9] 99.61911 99.61911 99.61911 99.61911

$lower
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [5]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'complete')
cat('ETS(A,N,N) with complete (nIterations=2):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,N,N) with complete (nIterations=2):
Parameters: 0 
$mean
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911
 [9] 99.61911 99.61911 99.61911 99.61911

$lower
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1


 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [6]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'two-stage', nIterations = 2)
cat('ETS(A,N,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)


ETS(A,N,N) with two-stage initialization (two-stage):
Parameters: 0 99.61911 
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911
 [9] 99.61911 99.61911 99.61911 99.61911


### Test 2: Local trend ETS(A,A,N)


In [7]:
np.random.seed(42)
n_points = 120
errors = np.random.normal(0, 10, n_points)
trend = np.random.normal(0.5, 2, n_points)
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series})


In [8]:
%%R -i ts_df

model <- adam(ts_df, model = "AAN", lags = c(12), initial = 'optimal')
forecast(model, h = 12)[1:3] 


$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 164.7969 165.3342 165.8714 166.4087 166.9460 167.4833 168.0206 168.5578
 [9] 169.0951 169.6324 170.1697 170.7070

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [9]:
%%R -i ts_df

model <- adam(ts_df, model = "AAN", lags = c(12), initial = 'complete')
cat('ETS(A,A,N) with complete initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] 


ETS(A,A,N) with complete initial:
Alpha, Beta, Gamma: 0.2007042 0 NA 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.2646 165.8909 166.5171 167.1433 167.7696 168.3958 169.0220 169.6483
 [9] 170.2745 170.9008 171.5270 172.1532

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [10]:
%%R -i ts_df

model <- adam(ts_df, model = "AAN", lags = c(12), initial = 'two-stage', nIterations = 2)
cat('ETS(A,A,N) with two-stage initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,N) with two-stage initial:
Alpha, Beta, Gamma: 0.4521797 0 57.65476 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.4278 165.4269 165.4260 165.4251 165.4242 165.4233 165.4224 165.4215
 [9] 165.4206 165.4196 165.4187 165.4178

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 3: Seasonal data ETS(A,A,A)


In [11]:
np.random.seed(42)
n_points = 120
errors = (1+np.random.normal(0, 0.1, n_points))
trend = np.random.normal(0.5, 2, n_points)
seasonal_sd = 0.2
seasonal_pattern = np.exp(np.random.normal(0, seasonal_sd, 12))
seasonal_pattern = seasonal_pattern / np.mean(seasonal_pattern)
time_series = np.zeros(n_points)
time_series[0] = 200 * seasonal_pattern[0] * errors[0]
for i in range(n_points-1):
    time_series[i+1] = ((time_series[i] / seasonal_pattern[(i) % 12]-trend[i]) * errors[i] ** (0.1-1) + trend[i+1]) * seasonal_pattern[(i+1) % 12] * errors[i+1]
ts_df = pd.DataFrame({'value': time_series})


In [12]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(A,A,A) with optimal initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with optimal initial:
Alpha, Beta, Gamma: 0.1262427 0.00535913 0.003895228 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 146.1119 158.7196 172.5692 204.7922 126.5985 164.4469 153.0418 150.9850
 [9] 240.4869 182.6033 136.5388 206.7759

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [13]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'backcasting')
cat('ETS(A,A,A) with backcasting initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with backcasting initial:
Alpha, Beta, Gamma: 0.1046893 0 0.07075804 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 143.8172 157.3988 172.9862 205.9623 125.8152 163.5787 154.9913 148.4720
 [9] 239.7151 180.6143 134.2696 203.4392

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [14]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'complete')
cat('ETS(A,A,A) with complete initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with complete initial:
Alpha, Beta, Gamma: 0.1046893 0 0.07075804 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 143.8172 157.3988 172.9862 205.9623 125.8152 163.5787 154.9913 148.4720
 [9] 239.7151 180.6143 134.2696 203.4392

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [15]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'two-stage', nIterations = 2)
cat('ETS(A,A,A) with two-stage initialization (two-stage):\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with two-stage initialization (two-stage):
Alpha, Beta, Gamma: 0.2096556 0 0.0002382092 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 150.3137 164.8220 179.0409 210.5942 133.0595 170.5958 162.4326 155.6900
 [9] 246.9474 187.2108 142.3292 213.8256

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 4: Damped trend ETS(A,Ad,N)


In [16]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'optimal')
cat('ETS(A,Ad,N) with optimal initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,Ad,N) with optimal initial:
Parameters: 0 0 0.9453906 214.7991 -2.425532 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 172.8556 172.8530 172.8506 172.8483 172.8461 172.8441 172.8422 172.8403
 [9] 172.8386 172.8370 172.8354 172.8339

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [17]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'backcasting')
cat('ETS(A,Ad,N) with backcasting initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,Ad,N) with backcasting initial:
Parameters: 0.002401592 0 1 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 167.3903 167.2213 167.0523 166.8832 166.7142 166.5452 166.3762 166.2072
 [9] 166.0382 165.8691 165.7001 165.5311

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [18]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'two-stage', nIterations = 2)
cat('ETS(A,Ad,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,Ad,N) with two-stage initialization (two-stage):
Parameters: 0 0 0.2824486 178.9303 -0.1896282 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 178.8557 178.8557 178.8557 178.8557 178.8557 178.8557 178.8557 178.8557
 [9] 178.8557 178.8557 178.8557 178.8557

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 5: Multiplicative error ETS(M,N,N)


In [19]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(M,N,N) with optimal initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,N,N) with optimal initial:
Parameters: 0.03293295 189.757 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 174.2432 174.2432 174.2432 174.2432 174.2432 174.2432 174.2432 174.2432
 [9] 174.2432 174.2432 174.2432 174.2432

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [20]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'backcasting')
cat('ETS(M,N,N) with backcasting initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,N,N) with backcasting initial:
Parameters: 0.03312988 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 174.1972 174.1972 174.1972 174.1972 174.1972 174.1972 174.1972 174.1972
 [9] 174.1972 174.1972 174.1972 174.1972

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [21]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'two-stage', nIterations = 2)
cat('ETS(M,N,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,N,N) with two-stage initialization (two-stage):
Parameters: 0.03298887 1058.789 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 174.2383 174.2383 174.2383 174.2383 174.2383 174.2383 174.2383 174.2383
 [9] 174.2383 174.2383 174.2383 174.2383

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 6: Multiplicative seasonal ETS(M,A,M)


In [22]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(M,A,M) with optimal initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with optimal initial:
Alpha, Beta, Gamma: 0.1144016 0.002874298 0.0009683954 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 141.9179 159.2557 176.0219 201.6552 126.9609 165.3692 155.2566 150.9177
 [9] 235.2195 178.3706 136.5630 206.6802

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [23]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'backcasting')
cat('ETS(M,A,M) with backcasting initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with backcasting initial:
Alpha, Beta, Gamma: 0.09886931 0 0.06262707 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 145.5738 158.3566 172.9111 203.5327 128.4125 163.9231 155.7289 149.5942
 [9] 236.1335 179.8870 136.0232 200.8808

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [24]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'complete')
cat('ETS(M,A,M) with complete initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with complete initial:
Alpha, Beta, Gamma: 0.09886931 0 0.06262707 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 145.5738 158.3566 172.9111 203.5327 128.4125 163.9231 155.7289 149.5942
 [9] 236.1335 179.8870 136.0232 200.8808

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [25]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'two-stage', nIterations = 2)
cat('ETS(M,A,M) with two-stage initialization (two-stage):\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with two-stage initialization (two-stage):
Alpha, Beta, Gamma: 0.1686831 0 0.0004969619 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 150.0277 164.8952 181.7984 208.9602 134.1334 171.3282 162.4049 158.5939
 [9] 245.9741 188.2698 144.2359 214.6856

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



## Summary

Two-stage initialization successfully:
- Runs a two-stage backcasting model first
- Extracts parameters (persistence, damping, ARMA)
- Extracts and normalizes initial states
- Uses these as starting values for optimization
- Produces results that combine benefits of both approaches
